# Explore Real-World Data with Pandas

This lab teaches you advanced Pandas techniques for analyzing complex, real-world datasets. You'll work with a realistic housing market dataset and practice advanced filtering, grouping, pivoting, and time-series analysis.

## Learning Objectives
By the end of this lab, you will be able to:
- Create and work with comprehensive, multi-table datasets
- Apply advanced filtering techniques for complex business questions
- Master grouping and aggregation for data summarization
- Use pivot tables and cross-tabulation for multi-dimensional analysis
- Perform time series analysis to identify trends and patterns
- Extract actionable business insights from complex data relationships

## Section 1: Setting Up a Comprehensive Dataset

In real-world data analysis, you often need to work with several related datasets at once. For example, analyzing the housing market means combining property details, sales transactions, and market trends.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducible results
np.random.seed(42)

print("=== CREATING COMPREHENSIVE HOUSING MARKET DATASET ===")

=== CREATING COMPREHENSIVE HOUSING MARKET DATASET ===


In [2]:
%time

CPU times: user 5 μs, sys: 2 μs, total: 7 μs
Wall time: 13.8 μs


In [3]:
# Create base property data with realistic characteristics
n_properties = 1000
property_ids = range(10001, 10001 + n_properties)

# Property characteristics follow realistic distributions
square_footage = np.random.normal(2200, 600, n_properties).astype(int)
square_footage = np.clip(square_footage, 800, 5000)

bedrooms = np.random.choice([2, 3, 4, 5, 6], n_properties, p=[0.15, 0.35, 0.35, 0.12, 0.03])
bathrooms = bedrooms + np.random.choice([-0.5, 0, 0.5, 1], n_properties, p=[0.1, 0.5, 0.3, 0.1])
bathrooms = np.clip(bathrooms, 1, 5)

current_year = 2024
ages = np.random.exponential(12, n_properties).astype(int)
ages = np.clip(ages, 0, 80)
year_built = current_year - ages

property_types = np.random.choice(
    ['Single Family', 'Townhouse', 'Condo', 'Duplex'], 
    n_properties, p=[0.6, 0.2, 0.15, 0.05]
)

neighborhoods = np.random.choice(
    ['Downtown', 'Suburban North', 'Suburban South', 'Riverside', 
     'Historic District', 'New Development', 'Industrial Area'], 
    n_properties, p=[0.1, 0.25, 0.2, 0.15, 0.1, 0.15, 0.05]
)

school_districts = np.random.choice(
    ['District A', 'District B', 'District C', 'District D'], 
    n_properties, p=[0.3, 0.3, 0.25, 0.15]
)

has_garage = np.random.choice([True, False], n_properties, p=[0.75, 0.25])
has_pool = np.random.choice([True, False], n_properties, p=[0.2, 0.8])
has_fireplace = np.random.choice([True, False], n_properties, p=[0.4, 0.6])
recently_renovated = np.random.choice([True, False], n_properties, p=[0.15, 0.85])

# Create properties DataFrame
properties_df = pd.DataFrame({
    'property_id': property_ids,
    'square_feet': square_footage,
    'bedrooms': bedrooms,
    'bathrooms': bathrooms,
    'year_built': year_built,
    'property_type': property_types,
    'neighborhood': neighborhoods,
    'school_district': school_districts,
    'has_garage': has_garage,
    'has_pool': has_pool,
    'has_fireplace': has_fireplace,
    'recently_renovated': recently_renovated
})

print(f"✓ Created properties dataset: {len(properties_df):,} properties")
print(f"\nSample property data:")
print(properties_df.head())
%time

✓ Created properties dataset: 1,000 properties

Sample property data:
   property_id  square_feet  bedrooms  bathrooms  year_built  property_type  \
0        10001         2498         3        3.0        2012  Single Family   
1        10002         2117         2        1.5        2024  Single Family   
2        10003         2588         4        4.0        2019  Single Family   
3        10004         3113         4        4.0        2005  Single Family   
4        10005         2059         2        2.5        2018  Single Family   

        neighborhood school_district  has_garage  has_pool  has_fireplace  \
0     Suburban South      District B        True     False          False   
1          Riverside      District A        True     False          False   
2     Suburban South      District C        True      True          False   
3           Downtown      District B        True     False           True   
4  Historic District      District C        True     False           T

In [4]:
# Create sales transaction data
print("\nCreating sales transaction data...")

start_date = datetime(2019, 1, 1)
end_date = datetime(2024, 6, 30)
date_range = (end_date - start_date).days

sales_data = []
transaction_id = 50001

for prop_id in property_ids:
    num_sales = np.random.choice([0, 1, 2, 3], p=[0.3, 0.5, 0.15, 0.05])
    property_info = properties_df[properties_df['property_id'] == prop_id].iloc[0]
    
    for sale_num in range(num_sales):
        days_offset = np.random.randint(0, date_range)
        sale_date = start_date + timedelta(days=days_offset)
        
        # Calculate realistic price using multiple factors
        base_price = 120 * property_info['square_feet']
        
        neighborhood_multipliers = {
            'Downtown': 1.4, 'Riverside': 1.25, 'Historic District': 1.2,
            'New Development': 1.15, 'Suburban North': 1.1, 'Suburban South': 1.05,
            'Industrial Area': 0.85
        }
        price_multiplier = neighborhood_multipliers.get(property_info['neighborhood'], 1.0)
        
        feature_bonus = 0
        if property_info['has_garage']: feature_bonus += 15000
        if property_info['has_pool']: feature_bonus += 25000
        if property_info['has_fireplace']: feature_bonus += 8000
        if property_info['recently_renovated']: feature_bonus += 20000
        
        district_multipliers = {'District A': 1.1, 'District B': 1.05, 
                              'District C': 1.0, 'District D': 0.95}
        district_multiplier = district_multipliers.get(property_info['school_district'], 1.0)
        
        age_at_sale = 2024 - property_info['year_built'] + (sale_date.year - 2024)
        age_penalty = max(0, age_at_sale * 800)
        
        years_since_2019 = (sale_date.year - 2019) + (sale_date.month - 1) / 12
        market_trend = 1.0 + (years_since_2019 * 0.06)
        
        calculated_price = (base_price * price_multiplier * district_multiplier + 
                          feature_bonus - age_penalty) * market_trend
        
        final_price = calculated_price * np.random.normal(1.0, 0.12)
        final_price = max(50000, round(final_price, -3))
        
        days_on_market = max(1, int(np.random.exponential(45)))
        
        sales_data.append({
            'transaction_id': transaction_id,
            'property_id': prop_id,
            'sale_date': sale_date,
            'sale_price': final_price,
            'days_on_market': days_on_market,
            'listing_price': final_price * np.random.uniform(1.0, 1.15),
            'seller_type': np.random.choice(['Individual', 'Developer', 'Bank'], p=[0.7, 0.2, 0.1])
        })
        transaction_id += 1

sales_df = pd.DataFrame(sales_data)
sales_df['listing_price'] = sales_df['listing_price'].round(-3)

print(f"✓ Created sales dataset: {len(sales_df):,} transactions")
print(f"\nSample sales data:")
print(sales_df.head())


Creating sales transaction data...
✓ Created sales dataset: 943 transactions

Sample sales data:
   transaction_id  property_id  sale_date  sale_price  days_on_market  \
0           50001        10001 2021-05-30    349000.0              19   
1           50002        10002 2022-06-20    350000.0               1   
2           50003        10004 2022-03-18    715000.0              15   
3           50004        10005 2021-11-24    407000.0              41   
4           50005        10005 2022-01-09    396000.0               1   

   listing_price seller_type  
0       355000.0  Individual  
1       353000.0        Bank  
2       721000.0  Individual  
3       435000.0  Individual  
4       437000.0  Individual  


**Exercise 1.1:** Explore the dataset structure:
1. Check the data types and basic statistics for each DataFrame
2. Verify that all properties in the sales data exist in the properties data
3. Find the property that has sold the most times
4. Calculate the overall price range across all sales

In [5]:
# Your exploration code here


## Section 2: Advanced Filtering and Selection Techniques

Complex datasets need more than simple filters to find useful data. Here, you'll learn advanced Pandas filtering to handle realistic scenarios.

In [6]:
print("=== ADVANCED FILTERING AND SELECTION TECHNIQUES ===")

# 1. Complex Boolean Filtering with Multiple Conditions
luxury_criteria = (
    (properties_df['square_feet'] > 3000) &
    (properties_df['has_pool'] == True) &
    (properties_df['has_garage'] == True) &
    (properties_df['neighborhood'].isin(['Downtown', 'Riverside', 'Historic District']))
)

luxury_properties = properties_df[luxury_criteria]
print(f"Luxury properties found: {len(luxury_properties)}")
print(f"Neighborhoods: {luxury_properties['neighborhood'].value_counts().to_dict()}")

# 2. Using query() method for readable complex conditions
starter_homes = properties_df.query(
    "bedrooms >= 2 and bedrooms <= 3 and "
    "bathrooms >= 1.5 and bathrooms <= 2.5 and "
    "square_feet >= 1200 and square_feet <= 2000 and "
    "property_type == 'Single Family'"
)

print(f"\nStarter homes identified: {len(starter_homes)}")
print(f"Average square feet: {starter_homes['square_feet'].mean():.0f}")

=== ADVANCED FILTERING AND SELECTION TECHNIQUES ===
Luxury properties found: 5
Neighborhoods: {'Historic District': 3, 'Riverside': 1, 'Downtown': 1}

Starter homes identified: 43
Average square feet: 1640


In [7]:
properties_df.head()

,property_id,square_feet,bedrooms,bathrooms,year_built,property_type,neighborhood,school_district,has_garage,has_pool,has_fireplace,recently_renovated
0,10001,2498,3,3.0,2012,Single Family,Suburban South,District B,True,False,False,False
1,10002,2117,2,1.5,2024,Single Family,Riverside,District A,True,False,False,False
2,10003,2588,4,4.0,2019,Single Family,Suburban South,District C,True,True,False,True
3,10004,3113,4,4.0,2005,Single Family,Downtown,District B,True,False,True,False
4,10005,2059,2,2.5,2018,Single Family,Historic District,District C,True,False,True,False


In [8]:
sales_df.head()

,transaction_id,property_id,sale_date,sale_price,days_on_market,listing_price,seller_type
0,50001,10001,2021-05-30,349000.0,19,355000.0,Individual
1,50002,10002,2022-06-20,350000.0,1,353000.0,Bank
2,50003,10004,2022-03-18,715000.0,15,721000.0,Individual
3,50004,10005,2021-11-24,407000.0,41,435000.0,Individual
4,50005,10005,2022-01-09,396000.0,1,437000.0,Individual


In [9]:
# 3. Multi-table Filtering with Strategic Merges
recent_expensive_sales = sales_df[
    (sales_df['sale_date'] >= '2022-01-01') &
    (sales_df['sale_price'] >= 500000)
]
recent_expensive_sales.head()

,transaction_id,property_id,sale_date,sale_price,days_on_market,listing_price,seller_type
2,50003,10004,2022-03-18,715000.0,15,721000.0,Individual
7,50008,10008,2024-01-25,505000.0,40,561000.0,Bank
41,50042,10035,2022-11-02,540000.0,18,560000.0,Individual
65,50066,10055,2024-04-24,681000.0,100,774000.0,Individual
92,50093,10082,2023-09-15,565000.0,11,629000.0,Individual


In [18]:
# 3. Multi-table Filtering with Strategic Merges

########### MERGE ######################################
expensive_with_details = recent_expensive_sales.merge(
    properties_df, on='property_id', how='left'
)
expensive_with_details.head()

,transaction_id,property_id,sale_date,sale_price,days_on_market,listing_price,seller_type,square_feet,bedrooms,bathrooms,year_built,property_type,neighborhood,school_district,has_garage,has_pool,has_fireplace,recently_renovated
0,50003,10004,2022-03-18,715000.0,15,721000.0,Individual,3113,4,4.0,2005,Single Family,Downtown,District B,True,False,True,False
1,50008,10008,2024-01-25,505000.0,40,561000.0,Bank,2660,4,4.0,2020,Single Family,New Development,District C,False,False,False,False
2,50042,10035,2022-11-02,540000.0,18,560000.0,Individual,2693,4,4.0,2013,Single Family,Suburban North,District A,True,False,False,False
3,50066,10055,2024-04-24,681000.0,100,774000.0,Individual,2818,5,5.0,2022,Townhouse,Historic District,District A,True,False,True,False
4,50093,10082,2023-09-15,565000.0,11,629000.0,Individual,2414,4,4.5,2022,Condo,Suburban North,District C,True,False,False,False


In [12]:
# 3. Multi-table Filtering with Strategic Merges


print(f"High-value recent sales: {len(expensive_with_details)}")
print(f"Average sale price: ${expensive_with_details['sale_price'].mean():,.0f}")
if len(expensive_with_details) > 0:
    print(f"Most common neighborhood: {expensive_with_details['neighborhood'].mode()[0]}")

# 4. Time-based Filtering for Seasonal Analysis
spring_sales = sales_df[sales_df['sale_date'].dt.month.isin([3, 4, 5])]
summer_sales = sales_df[sales_df['sale_date'].dt.month.isin([6, 7, 8])]

print(f"\nSpring sales (Mar-May): {len(spring_sales)}")
print(f"Summer sales (Jun-Aug): {len(summer_sales)}")
print(f"Spring avg price: ${spring_sales['sale_price'].mean():,.0f}")
print(f"Summer avg price: ${summer_sales['sale_price'].mean():,.0f}")

High-value recent sales: 90
Average sale price: $574,289
Most common neighborhood: Riverside

Spring sales (Mar-May): 232
Summer sales (Jun-Aug): 208
Spring avg price: $383,552
Summer avg price: $372,091


**Exercise 2.1:** Practice advanced filtering:
1. Properties with pools in suburban areas built after 2010
2. Sales in 2023 where the property sold for more than the listing price
3. Recently renovated properties that took longer than 60 days to sell
4. Properties in Districts A or B that have both a garage and fireplace

In [ ]:
# Your filtering practice here


## Section 3: Grouping and Aggregation Mastery

Grouping and aggregation help you summarize large datasets to find patterns and trends. These techniques let you turn raw data into clear business insights.

In [13]:
print("=== GROUPING AND AGGREGATION MASTERY ===")

# 1. Multi-Function Aggregations
neighborhood_analysis = sales_df.merge(properties_df, on='property_id').groupby('neighborhood').agg({
    'sale_price': ['count', 'mean', 'median', 'std', 'min', 'max'],
    'days_on_market': ['mean', 'median'],
    'square_feet': 'mean',
    'bedrooms': 'mean'
}).round(2)

neighborhood_analysis.head()

=== GROUPING AND AGGREGATION MASTERY ===


sale_price                                            \
                       count       mean    median        std       min   
neighborhood                                                             
Downtown                  82  429280.49  423000.0  133987.31  181000.0   
Historic District         97  399443.30  392000.0  112758.03  123000.0   
Industrial Area           52  280576.92  281500.0   74109.71  112000.0   
New Development          145  375048.28  359000.0  111938.37  139000.0   
Riverside                143  409615.38  403000.0  111087.32  198000.0   

                            days_on_market        square_feet bedrooms  
                        max           mean median        mean     mean  
neighborhood                                                            
Downtown           716000.0          39.80   28.0     2045.82     3.57  
Historic District  658000.0          49.78   47.0     2187.77     3.36  
Industrial Area    448000.0          48.04   36.5     2219.83     3.37  
New Development    717000.0          56.87   37.0     2197.79     3.66  
Riverside          687000.0          44.03   31.0     2183.44     3.39

In [14]:
neighborhood_analysis.columns = [f"{col[0]}_{col[1]}" for col in neighborhood_analysis.columns]
neighborhood_analysis = neighborhood_analysis.sort_values('sale_price_mean', ascending=False)

print("Neighborhood Analysis (Top 5 by average price):")
print(neighborhood_analysis.head())

Neighborhood Analysis (Top 5 by average price):
                   sale_price_count  sale_price_mean  sale_price_median  \
neighborhood                                                              
Downtown                         82        429280.49           423000.0   
Riverside                       143        409615.38           403000.0   
Historic District                97        399443.30           392000.0   
Suburban North                  207        375666.67           372000.0   
New Development                 145        375048.28           359000.0   

                   sale_price_std  sale_price_min  sale_price_max  \
neighborhood                                                        
Downtown                133987.31        181000.0        716000.0   
Riverside               111087.32        198000.0        687000.0   
Historic District       112758.03        123000.0        658000.0   
Suburban North          104611.67        150000.0        642000.0   
New Developm

In [15]:
# 2. Custom Aggregation Functions
def price_range_description(prices):
    return f"${prices.min():,.0f} - ${prices.max():,.0f}"

def market_performance(prices):
    return (prices.std() / prices.mean()) * 100

neighborhood_custom = sales_df.merge(properties_df, on='property_id').groupby('neighborhood').agg({
    'sale_price': [price_range_description, market_performance, 'count']
}).round(2)

neighborhood_custom.columns = ['price_range', 'price_volatility', 'sales_count']
print("\nCustom neighborhood metrics:")
print(neighborhood_custom.sort_values('price_volatility'))


Custom neighborhood metrics:
                           price_range  price_volatility  sales_count
neighborhood                                                         
Industrial Area    $112,000 - $448,000             26.41           52
Riverside          $198,000 - $687,000             27.12          143
Suburban North     $150,000 - $642,000             27.85          207
Historic District  $123,000 - $658,000             28.23           97
Suburban South     $146,000 - $734,000             29.01          185
New Development    $139,000 - $717,000             29.85          145
Downtown           $181,000 - $716,000             31.21           82


In [17]:
# 3. Hierarchical Grouping
multi_level_analysis = sales_df.merge(properties_df, on='property_id').groupby([
    'neighborhood', 'property_type'
]).agg({
    'sale_price': ['count', 'mean'],
    'days_on_market': 'mean',
    'square_feet': 'mean'
}).round(2)

#type(multi_level_analysis)
multi_level_analysis.head()

sale_price            days_on_market  \
                                     count       mean           mean   
neighborhood      property_type                                        
Downtown          Condo                  4  363000.00          70.75   
                  Duplex                 2  337000.00          16.50   
                  Single Family         53  425207.55          43.02   
                  Townhouse             23  458217.39          29.04   
Historic District Condo                 14  411357.14          39.79   

                                square_feet  
                                       mean  
neighborhood      property_type              
Downtown          Condo             1672.75  
                  Duplex            1640.50  
                  Single Family     1982.21  
                  Townhouse         2292.52  
Historic District Condo             2131.50

In [19]:
multi_level_analysis.columns = ['sales_count', 'avg_price', 'avg_days_market', 'avg_sqft']
print("\nTop neighborhood-property type combinations by sales volume:")
print(multi_level_analysis.sort_values('sales_count', ascending=False).head(10))


Top neighborhood-property type combinations by sales volume:
                                 sales_count  avg_price  avg_days_market  \
neighborhood      property_type                                            
Suburban North    Single Family          128  370476.56            41.80   
Suburban South    Single Family          116  354456.90            47.83   
Riverside         Single Family           98  412755.10            46.51   
New Development   Single Family           82  376268.29            53.07   
Historic District Single Family           62  397387.10            49.24   
Downtown          Single Family           53  425207.55            43.02   
Suburban North    Townhouse               44  386363.64            45.93   
Suburban South    Townhouse               41  330902.44            50.49   
New Development   Townhouse               38  365052.63            66.13   
Suburban North    Condo                   31  382709.68            33.45   

                         

**Exercise 3.1:** Practice grouping and aggregation:
1. Calculate average price per square foot by property type
2. Find the school district with the fastest average selling time
3. Group by number of bedrooms and show price statistics
4. Create a multi-level grouping by year and season

In [ ]:
# Your grouping and aggregation practice here


## Section 4: Pivot Tables and Cross-tabulation Analysis

Pivot tables let you quickly summarize and compare data across multiple categories. They help you explore how different factors relate to each other.

In [20]:
print("=== PIVOT TABLES AND CROSS-TABULATION ANALYSIS ===")

pivot_data = sales_df.merge(properties_df, on='property_id')
pivot_data['sale_year'] = pivot_data['sale_date'].dt.year

# 1. Basic Pivot Table Analysis
basic_pivot = pivot_data.pivot_table(
    values='sale_price',
    index='neighborhood',
    columns='property_type',
    aggfunc=['count', 'mean'],
    fill_value=0
)

basic_pivot.columns = [f"{col[1]}_{col[0]}" for col in basic_pivot.columns]

print("Sales count by neighborhood and property type:")
count_columns = [col for col in basic_pivot.columns if 'count' in col]
print(basic_pivot[count_columns])

=== PIVOT TABLES AND CROSS-TABULATION ANALYSIS ===
Sales count by neighborhood and property type:
                   Condo_count  Duplex_count  Single Family_count  \
neighborhood                                                        
Downtown                     4             2                   53   
Historic District           14             9                   62   
Industrial Area             16             4                   26   
New Development             18             7                   82   
Riverside                   19             3                   98   
Suburban North              31             4                  128   
Suburban South              20             8                  116   

                   Townhouse_count  
neighborhood                        
Downtown                        23  
Historic District               12  
Industrial Area                  6  
New Development                 38  
Riverside                       23  
Suburban North       

In [21]:
# 2. Feature Impact Analysis
feature_pivot = pivot_data.pivot_table(
    values='sale_price',
    index=['has_garage', 'has_pool'],
    columns='has_fireplace',
    aggfunc=['count', 'mean'],
    fill_value=0,
    margins=True
)

print("\nProperty feature combinations impact on price:")
print(feature_pivot.round(2))


Property feature combinations impact on price:
                    count                 mean                      
has_fireplace       False True  All      False       True        All
has_garage has_pool                                                 
False      False      131   93  224  372244.27  359279.57  366861.61
           True        33    6   39  409363.64  309333.33  393974.36
True       False      317  209  526  372823.34  379693.78  375553.23
           True        70   52  122  396728.57  415942.31  404918.03
All                   551  360  911  377911.07  378483.33  378137.21


In [22]:
# 3. Market Segment Analysis
def price_segment(price):
    if price >= 700000:
        return 'Ultra-Premium'
    elif price >= 500000:
        return 'Premium'
    elif price >= 350000:
        return 'Mid-Market'
    elif price >= 200000:
        return 'Affordable'
    else:
        return 'Entry-Level'

pivot_data['price_segment'] = pivot_data['sale_price'].apply(price_segment)

segment_distribution = pd.crosstab(
    pivot_data['neighborhood'],
    pivot_data['price_segment'],
    margins=True,
    normalize='index'
)

print("\nPrice segment distribution by neighborhood (%):")
print((segment_distribution * 100).round(1))


Price segment distribution by neighborhood (%):
price_segment      Affordable  Entry-Level  Mid-Market  Premium  Ultra-Premium
neighborhood                                                                  
Downtown                 29.3          1.2        40.2     28.0            1.2
Historic District        32.0          2.1        48.5     17.5            0.0
Industrial Area          71.2         13.5        15.4      0.0            0.0
New Development          40.7          4.1        40.7     13.8            0.7
Riverside                35.0          0.7        41.3     23.1            0.0
Suburban North           39.1          4.3        44.0     12.6            0.0
Suburban South           46.5          5.9        38.9      8.1            0.5
All                      40.4          4.1        40.5     14.7            0.3


**Exercise 4.1:** Create your own pivot tables:
1. Sales volume by year and property type
2. Average days on market by school district and number of bedrooms
3. Cross-tabulation of property features
4. Price per square foot analysis by neighborhood and year built ranges

In [ ]:
# Your pivot table practice here


## Section 5: Time Series Analysis and Trends

Time series analysis helps you understand how things change over time, such as market trends and seasonal patterns.

In [ ]:
print("=== TIME SERIES ANALYSIS AND TRENDS ===")

ts_data = sales_df.merge(properties_df, on='property_id').copy()
ts_data['sale_date'] = pd.to_datetime(ts_data['sale_date'])
ts_data = ts_data.sort_values('sale_date')

# 1. Time Series Data Preparation
monthly_metrics = ts_data.groupby(pd.Grouper(key='sale_date', freq='M')).agg({
    'sale_price': ['count', 'mean', 'median', 'std'],
    'days_on_market': 'mean',
    'square_feet': 'mean',
    'property_id': 'nunique'
}).round(2)

monthly_metrics.columns = ['sales_volume', 'avg_price', 'median_price', 'price_volatility', 
                          'avg_days_market', 'avg_size', 'unique_properties']

print("Monthly market metrics (last 12 months):")
print(monthly_metrics.tail(12))

In [ ]:
# 2. Trend Analysis
monthly_metrics['price_trend_3m'] = monthly_metrics['avg_price'].rolling(window=3).mean()
monthly_metrics['price_yoy_change'] = monthly_metrics['avg_price'].pct_change(periods=12) * 100
monthly_metrics['volume_yoy_change'] = monthly_metrics['sales_volume'].pct_change(periods=12) * 100

print("\nMarket trends with year-over-year changes:")
trend_cols = ['avg_price', 'price_trend_3m', 'price_yoy_change', 'sales_volume', 'volume_yoy_change']
print(monthly_metrics[trend_cols].tail(12).round(2))

In [ ]:
# 3. Seasonal Analysis
ts_data['month'] = ts_data['sale_date'].dt.month
ts_data['year'] = ts_data['sale_date'].dt.year

seasonal_patterns = ts_data.groupby('month').agg({
    'sale_price': ['count', 'mean'],
    'days_on_market': 'mean'
}).round(2)

seasonal_patterns.columns = ['monthly_volume', 'monthly_avg_price', 'monthly_days_market']

annual_avg_price = ts_data['sale_price'].mean()
annual_avg_volume = ts_data.groupby('year')['sale_price'].count().mean()

seasonal_patterns['price_seasonal_index'] = (seasonal_patterns['monthly_avg_price'] / annual_avg_price) * 100
seasonal_patterns['volume_seasonal_index'] = (seasonal_patterns['monthly_volume'] / annual_avg_volume) * 100

print("\nSeasonal patterns (seasonal index: 100 = average):")
print(seasonal_patterns[['price_seasonal_index', 'volume_seasonal_index']].round(1))

peak_price_month = seasonal_patterns['price_seasonal_index'].idxmax()
peak_volume_month = seasonal_patterns['volume_seasonal_index'].idxmax()

month_names = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
               7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

print(f"\nPeak price month: {month_names[peak_price_month]}")
print(f"Peak volume month: {month_names[peak_volume_month]}")

In [ ]:
# 4. Trend Visualization
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(monthly_metrics.index, monthly_metrics['avg_price'], label='Monthly Avg Price', linewidth=2)
plt.plot(monthly_metrics.index, monthly_metrics['price_trend_3m'], label='3-Month Trend', linewidth=2, alpha=0.7)
plt.title('Housing Market Price Trends Over Time')
plt.ylabel('Average Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(monthly_metrics.index, monthly_metrics['sales_volume'], label='Monthly Sales Volume', linewidth=2, color='orange')
plt.title('Housing Market Sales Volume Over Time')
plt.ylabel('Number of Sales')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Exercise 5.1:** Practice time series analysis:
1. Calculate quarterly trends instead of monthly
2. Find the year with the highest price growth
3. Identify any unusual spikes or drops in sales volume
4. Compare price trends between different neighborhoods

In [ ]:
# Your time series analysis practice here


## Section 6: Comprehensive Business Analysis

Now let's combine everything you've learned to generate comprehensive business insights.

In [ ]:
print("=== COMPREHENSIVE BUSINESS INSIGHTS ===")

# Market Performance Summary
current_date = monthly_metrics.index[-1]
current_price = monthly_metrics['avg_price'].iloc[-1]
yoy_change = monthly_metrics['price_yoy_change'].iloc[-1]
current_volume = monthly_metrics['sales_volume'].iloc[-1]

print(f"📊 MARKET STATUS SUMMARY (as of {current_date.strftime('%Y-%m')})")
print(f"   • Current Average Price: ${current_price:,.0f}")
print(f"   • Year-over-Year Change: {yoy_change:+.1f}%")
print(f"   • Monthly Sales Volume: {current_volume:.0f} transactions")

# Top performing segments
print(f"\n🏆 TOP PERFORMING SEGMENTS")
top_neighborhoods = neighborhood_analysis.head(3)
print(f"   • Highest Priced Neighborhoods:")
for idx, row in top_neighborhoods.iterrows():
    print(f"     - {idx}: ${row['sale_price_mean']:,.0f} avg (n={row['sale_price_count']:.0f})")

# Seasonal recommendations
print(f"\n📅 SEASONAL STRATEGY RECOMMENDATIONS")
print(f"   • Best price month: {month_names[peak_price_month]}")
print(f"   • Highest volume month: {month_names[peak_volume_month]}")

print(f"\n✅ Comprehensive analysis completed - Ready for strategic decision-making")

**Final Challenge:** Create your own comprehensive market analysis by combining techniques from all sections:

1. **Market Segmentation**: Define and analyze 3-4 market segments
2. **Competitive Analysis**: Compare performance across neighborhoods and property types
3. **Trend Forecasting**: Use historical data to make predictions
4. **Investment Strategy**: Identify specific opportunities
5. **Business Recommendations**: Provide actionable insights for stakeholders

In [ ]:
# Your comprehensive market analysis here


## Lab Summary and Reflection

Congratulations! You've completed an advanced exploration of real-world data using Pandas. In this lab, you've mastered techniques that are essential for professional data analysis.

### Key Skills Mastered

**Data Architecture**: You learned to work with multiple related datasets, understanding how real businesses structure their data across different tables and systems.

**Advanced Filtering**: You moved beyond basic indexing to complex boolean operations, query syntax, and multi-table filtering that answers sophisticated business questions.

**Aggregation Mastery**: You learned to summarize data using multiple aggregation functions, custom business metrics, and hierarchical grouping that reveals hidden patterns.

**Multi-dimensional Analysis**: Through pivot tables and cross-tabulation, you gained the ability to analyze data across multiple categories simultaneously.

**Temporal Analysis**: You developed skills to analyze how data changes over time, identifying trends, seasonal patterns, and market cycles.

### Why These Skills Matter

These techniques represent the core of professional data analysis. In real business environments, you'll regularly need to:
- Combine data from multiple sources to answer complex questions
- Filter large datasets to focus on specific segments or conditions
- Summarize and aggregate data to identify trends and patterns
- Create cross-sectional analyses that reveal relationships between variables
- Analyze temporal data to understand business cycles and predict future performance

### Real-World Applications

The skills you've learned apply far beyond real estate:
- **Retail**: Analyzing customer behavior, seasonal trends, and product performance
- **Finance**: Risk assessment, portfolio analysis, and market trend identification
- **Healthcare**: Patient outcome analysis, treatment effectiveness, and resource optimization
- **Manufacturing**: Quality control, production optimization, and supply chain analysis
- **Marketing**: Campaign effectiveness, customer segmentation, and conversion analysis

### Reflection Questions

1. **Integration**: How did combining multiple DataFrames change your ability to answer business questions?
2. **Complexity**: Which analysis technique did you find most powerful for revealing insights, and why?
3. **Business Value**: Can you think of a real business scenario where these techniques would provide competitive advantage?
4. **Methodology**: How has your approach to data analysis evolved from simple calculations to comprehensive business intelligence?

**Remember**: The goal isn't just to manipulate data—it's to extract actionable insights that drive better business decisions.